# Accessing OBIS through R: `robis` package

`robis` is the main gateway for OBIS data through R. You can learn more about `robis` in the [OBIS manual.]()

Hands-on outline:

1. Downloading occurrence records
2. Getting additional information about datasets
3. Download records for a particular region
4. Get species lists
5. Filtering data
6. Obtaining time series
7. Getting extended measurements

Before starting, we need to install a few packages on Google Colab:

In [ ]:
install.packages("bspm")
suppressMessages(bspm::enable())
install.packages("robis")
devtools::install_github("iobis/obistools")

### 1. Downloading occurrence records

To download records from OBIS we use the function `occurrence`. There are many arguments you can pass to download data for a specific species, taxonomic level or region.

We will start by getting data for three taxonomic entities:

<div style="display: flex; flex-direction: row; max-height: 200px; padding: 5px;">
<div>
<p>Species: <i>Lytechinus variegatus</i></p><img src="https://upload.wikimedia.org/wikipedia/commons/f/f5/Lytechinus_variegatus.jpg" height=200></img>
</div>
<div>
<p>Genus: <i>Lytechinus</i></p><img src="https://upload.wikimedia.org/wikipedia/commons/f/f5/Lytechinus_semituberculatus_12770656.jpg" height=200></img>
</div>
<div>
<p>Family: Toxopneustidae</p><img src="https://upload.wikimedia.org/wikipedia/commons/5/51/Toxopneustes_pileolus_%28Sea_urchin%29.jpg" height=200></img>
</div>
</div>

In [ ]:
library(robis)
library(dplyr)

lych_var <- occurrence("Lytechinus variegatus")

nrow(lych_var)
head(lych_var)

We can make a map of the records. For that, we can use a handy function from the `obistools` package.

In [ ]:
library(obistools)

plot_map(lych_var)

OBIS is different in that it was created and is optimized for marine data. It matches its taxonomic names (the species identity) with the World Register of Marine Species (WoRMS). Elych species (or any other taxonomic level) has a unique ID called **AphiaID**. You can see for example the entry for _Acanthurus varurgus_ [here.](https://www.marinespecies.org/aphia.php?p=taxdetails&id=159580)

This is important because species names can change over time, in case a species is reclassified into another group or when we discover that one species can be in fact multiple species.

The `occurrence` function also accepts taxonid. So, the same we did on the first cell, can be done with:

In [ ]:
# This is the same as
# lych_var <- occurrence("Lytechinus variegatus")
lych_var <- occurrence(taxonid = 367850)

Let's try the other taxonomic levels:

In [ ]:
lych_genus <- occurrence("Lytechinus")

nrow(lych_genus)
table(lych_genus$scientificName)

toxop_fam <- occurrence("Toxopneustidae")

nrow(toxop_fam)
table(toxop_fam$scientificName)

### 2. Getting dataset information

Records are organized in **datasets** that group data that was collected in a particular survey, study, monitoring, etc. We can get additional information about datasets using the `dataset` function (which can also be used to list all datasets for a specific `scientificName`).

In [ ]:
# Get the number of records by dataset
lych_var_ds <- lych_var |> 
    group_by(dataset_id) |> 
    summarise(records = n())

high_n <- lych_var_ds[order(lych_var_ds$records, descending = T), "dataset_id"]

ds_info <- dataset(datasetid = high_n)

ds_info

### 3. Downloading records for a region



### 4. Get list of species (checklist)

### 5. Filtering data

### 6. Time-series

### 7. Extended measurements